In [ ]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
from ner.llm_ner.ResultInstance import load_all_results, load_result
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2

In [ ]:

df_results = load_all_results(root_directory = "ner/saves/results/ontonote5/mistral-7b-v0.1/")

In [ ]:
df_to_show = df_results[['f1_mean', 'f1_conf_inter', 'prompt_technique',
       'few_shot_tecnique', 'nb_few_shots', 'precision', 'plus_plus']]

df_to_show[df_to_show['precision'] == '300']

## Show results for finetuned models

In [ ]:

df_results['finetuned'] = df_results['model'].map(lambda s : 'ft' in s)
df_results['nb_samples_tested'] = df_results['nb_test_run'] * df_results['len_data_test']
ft_results = df_results[df_results['finetuned']]
ft_results = ft_results[['f1_mean', 'f1_conf_inter', 'prompt_technique', 'model', 'noshots', 'few_shot_tecnique','nb_few_shots', 'nb_samples_tested']]
ft_results['model_base'] = ft_results['model'].map(lambda s : '-'.join(s.split('-')[:-4]))
ft_results['nb_train_samples'] = ft_results['model'].map(lambda s : s.split('-')[-2])
ft_results = ft_results.sort_values(['prompt_technique',	'few_shot_tecnique', 'f1_mean'], ascending=False)

In [ ]:
# ['f1_mean', 'f1_conf_inter', 'prompt_technique', 'few_shot_tecnique',
    #    'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples',
    #    'finetuned']
grouped = ft_results.groupby(['prompt_technique', 'few_shot_tecnique', 'noshots', 
       'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples'])
grouped 

idx_max = grouped['f1_mean'].idxmax()

ft_cleaned = ft_results.loc[idx_max].sort_values('nb_train_samples', ascending=False)

df = ft_cleaned#[ft_cleaned['nb_samples_tested'] == 500]
# Create dummy columns for 'few_shot_tecnique'
df.pivot(index=['model_base','prompt_technique','noshots',
        'nb_train_samples','nb_samples_tested'], columns='few_shot_tecnique', values= 'f1_mean')

## Show results for non finetuned models

In [ ]:
df_results['finetuned'] = df_results['model'].map(lambda s : 'ft' in s)
df_results['nb_samples_tested'] = df_results['nb_test_run'] * df_results['len_data_test']
ft_results = df_results[~df_results['finetuned']]
ft_results = ft_results[['f1_mean', 'f1_conf_inter', 'prompt_technique', 'model', 'noshots', 'few_shot_tecnique','nb_few_shots', 'nb_samples_tested']]
ft_results = ft_results.sort_values(['prompt_technique',	'few_shot_tecnique', 'f1_mean'], ascending=False)

In [ ]:
# ['f1_mean', 'f1_conf_inter', 'prompt_technique', 'few_shot_tecnique',
    #    'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples',
    #    'finetuned']
ft_results = ft_results[ft_results['model'] != 'none']
grouped = ft_results.groupby([ 'model', 'prompt_technique', 
       'nb_few_shots', 'nb_samples_tested','few_shot_tecnique', ])
grouped 

# idx_max = grouped['f1_mean'].idxmax()
# idx_max

# Create dummy columns for 'few_shot_tecnique'
ft_results.loc[idx_max].pivot(index=['model','prompt_technique', 'nb_few_shots','nb_samples_tested',
        ], columns='few_shot_tecnique', values= 'f1_mean')

## Showing prompts 

In [ ]:
LLMModel.show_prompts()

## Looking at results of confidence with the tagger

In [ ]:
from ner.utils import load

res = load ("ner/saves/results/conll2003_cleaned/mistral-7b-v0.1/multi_prompt-get-entities-tagger/sentence_3_None_1538_50_1_False.pkl")

cm, f1, precision, recall, y_true, y_pred, y_conf= res.res_insts[0].get_scores(with_y = True)
conf = []
for sent in res.res_insts[0].results:
    conf.extend([s[2] for s in sent])

len(y_conf), len(y_true), len(y_pred)
# [i for i, tag in enumerate(y_pred) if tag == 'None']
right = [y_true[i] == y_pred[i] for i in range(len(y_true))]
rand = [i for i in range(len(right))]

In [ ]:
from collections import Counter

# Example list
my_list = list(zip(y_conf, right))

# Use Counter to count occurrences
entry_counts = Counter(sorted(my_list))

# Print the counts
for entry, count in entry_counts.items():
    print(f'{entry}: {count} times')

In [ ]:
import seaborn as sns
df_conf = pd.DataFrame(np.array([right, y_conf]).T, columns = ['Right', 'Confidence'])

df_conf = pd.DataFrame(df_conf.groupby(['Right', 'Confidence'])['Right'].count()).rename(columns = {'Right' : "count"}).reset_index()
sns.set(rc={'figure.figsize':(10,10)})
order = ['high', 'medium-high', 'medium', 'medium-low', 'low']
sns.catplot(data=df_conf, x="Confidence", y="count", hue="Right", kind="bar",aspect=1.5, order = order)


## Analysing specific result

In [ ]:
model  = "mistral-7b-v0.1"
result = load_result(model, PT_OutputList.name(), FST_Sentence.name(), nb_few_shots= 10, with_precision = "10_True")

for r in result.res_insts :
    r.show_cm()
    r.analyse_results()

In [ ]:
r.f1

In [ ]:
from ner.Datasets.Conll2003Dataset import Conll2003Dataset
from ner.Datasets.MyDataset import MyDataset
data = MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)
[(d['text'], d['spans']) for d in data if str(d['spans']) == """[['Switzerland', 'LOC'], ["Alfonse D'Amato", 'PER'], ['U.S. Senate Banking Committee', 'ORG'], ['Poland', 'LOC'], ['Polish', 'MISC'], ['Swiss', 'MISC'], ['Poland', 'LOC']]"""]

In [ ]:
[(d['text'], d['spans']) for d in data if 'Hansenne' in d['text']]